# Segmenting and Clustering Neighborhoods in Toronto City

## Importing required libararies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge bs4 --yes # uncomment this line if you haven't completed the Foursquare API lab
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Rendering the Wikipedia page for neighbourhood data and converting to a clean data frame

In [2]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
#website_url

In [3]:
soup = BeautifulSoup(website_url)
#print(soup.prettify())

In [4]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#print(My_table)

In [5]:
rows = My_table.find_all('tr')
len(rows)

181

In [6]:
columns  = [cell.get_text().replace('\n','') for cell in rows[0].find_all('th')]
columns

['Postal Code', 'Borough', 'Neighbourhood']

In [7]:
neighborhood_data = pd.DataFrame(columns=columns)
for row in rows[1:]:
    cells = [cell.get_text().replace('\n','') for cell in row.find_all('td')]
    #print(cells)
    if cells[1] == "Not assigned":
        continue
    if cells[2] == "Not assigned":
        cells[2] = cells[1]
    neighborhood_data = neighborhood_data.append({columns[0]: cells[0],
                                          columns[1]: cells[1],
                                          columns[2]: cells[2]}, ignore_index=True)


In [8]:
neighborhood_data.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [9]:
neighborhood_data.shape

(103, 3)

## Pulling out the neighborhoods from each row if found merged 

In [10]:
neighborhood_data_new = pd.DataFrame(columns=columns)

In [11]:
#list(map(strip,neighborhood_data.loc[2]['Neighbourhood'].split(",")))

In [12]:
for index in range(neighborhood_data.shape[0]):
    row = neighborhood_data.loc[index]
    hoods = row['Neighbourhood'].split(",")
    #print(hoods)
    for hood in hoods:
        neighborhood_data_new = neighborhood_data_new.append({columns[0]: row[0],
                                          columns[1]: row[1],
                                          columns[2]: hood.strip()}, ignore_index=True)

In [13]:
neighborhood_data_new.head(15)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor
5,M6A,North York,Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park
7,M7A,Downtown Toronto,Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M9A,Etobicoke,Humber Valley Village


In [14]:
neighborhood_data_new.shape

(217, 3)

## Finding unique pairs of Borough and Neighborhood

In [15]:
neighborhood_data_new = neighborhood_data_new.groupby(['Borough','Neighbourhood']).count().reset_index().drop('Postal Code',axis=1)
print(neighborhood_data_new.shape)
neighborhood_data_new.head()

(209, 2)


,Borough,Neighbourhood
0,Central Toronto,Davisville
1,Central Toronto,Davisville North
2,Central Toronto,Deer Park
3,Central Toronto,Forest Hill North & West
4,Central Toronto,Forest Hill Road Park


In [16]:
print("Number of Boroughs =",len(neighborhood_data_new['Borough'].unique()))
print("Number of Neighbourhoods =",neighborhood_data_new.shape[0])

Number of Boroughs = 10
Number of Neighbourhoods = 209


In [17]:
neighborhood_data_new.groupby(['Borough']).count()

,Neighbourhood
Borough,
Central Toronto,17
Downtown Toronto,38
East Toronto,8
East York,7
Etobicoke,47
Mississauga,1
North York,32
Scarborough,38
West Toronto,13


### lets just work on the borough with maximum Neighborhoods

In [18]:
EBC = neighborhood_data_new[neighborhood_data_new['Borough']=='Etobicoke'].reset_index(drop=True)
EBC.head(10)

,Borough,Neighbourhood
0,Etobicoke,Albion Gardens
1,Etobicoke,Alderwood
2,Etobicoke,Beaumond Heights
3,Etobicoke,Bloordale Gardens
4,Etobicoke,Cloverdale
5,Etobicoke,Eringate
6,Etobicoke,Humber Bay
7,Etobicoke,Humber Bay Shores
8,Etobicoke,Humber Valley Village
9,Etobicoke,Humbergate


In [19]:
lat = []
lng = []
for index in range(EBC.shape[0]):
    address = EBC.iloc[index,1]+", Toronto"
    #print(address)
    location = Nominatim(user_agent="ny_explorer").geocode(address)
    try:
        lat.append(location.latitude)
        lng.append(location.longitude)
    except:
        lat.append(None)
        lng.append(None)
EBC['LAT'] = lat
EBC['Long'] = lng
EBC.head(10)

,Borough,Neighbourhood,LAT,Long
0,Etobicoke,Albion Gardens,43.741665,-79.584543
1,Etobicoke,Alderwood,43.601717,-79.545232
2,Etobicoke,Beaumond Heights,NaN,NaN
3,Etobicoke,Bloordale Gardens,43.635317,-79.563674
4,Etobicoke,Cloverdale,43.633637,-79.549745
5,Etobicoke,Eringate,43.662273,-79.576516
6,Etobicoke,Humber Bay,43.640046,-79.495028
7,Etobicoke,Humber Bay Shores,NaN,NaN
8,Etobicoke,Humber Valley Village,43.666472,-79.524314
9,Etobicoke,Humbergate,43.616944,-79.477612


In [20]:
EBC.shape

(47, 4)

In [21]:
EBC= EBC[~EBC['LAT'].isna()]
EBC.shape

(45, 4)

In [22]:
EBC.head()

,Borough,Neighbourhood,LAT,Long
0,Etobicoke,Albion Gardens,43.741665,-79.584543
1,Etobicoke,Alderwood,43.601717,-79.545232
3,Etobicoke,Bloordale Gardens,43.635317,-79.563674
4,Etobicoke,Cloverdale,43.633637,-79.549745
5,Etobicoke,Eringate,43.662273,-79.576516


#### Define Foursquare Credentials and Version


In [23]:
address = 'Etobicoke, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Etobicoke are 43.6435559, -79.5656326.


In [24]:
# create map of Etobicoke using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(EBC['LAT'], EBC['Long'], EBC['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [25]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXXXXXXXXXXXXXXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Your credentails:
CLIENT_ID: FRWSSHW342W3NLPU1JBJMSMPGWJXJQD0J00I3PP025XNNY52
CLIENT_SECRET:5GE0VNOW1JOSWOQCIM1UTUBZHBQL4AUHRIYUHFYRFM20RVCF


## Explore Neighborhoods in Etobicoke


In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# type your answer here

EBC_venues = getNearbyVenues(EBC['Neighbourhood'],EBC['LAT'], EBC['Long'],  radius=950)


Albion Gardens
Alderwood
Bloordale Gardens
Cloverdale
Eringate
Humber Bay
Humber Valley Village
Humbergate
Islington
Islington Avenue
Jamestown
King's Mill Park
Kingsview Village
Kingsway Park South East
Kingsway Park South West
Long Branch
Markland Wood
Martin Grove
Martin Grove Gardens
Mimico NE
Mimico NW
Mimico South
Montgomery Road
Mount Olive
New Toronto
Northwest
Old Burnhamthorpe
Old Mill North
Old Mill South
Princess Gardens
Richview Gardens
Royal York South East
Royal York South West
Silverstone
South Steeles
South of Bloor
St. Phillips
Sunnylea
The Kingsway
The Queensway East
The Queensway West
Thistletown
West Deane Park
West Humber - Clairville
Westmount


#### Let's check the size of the resulting dataframe


In [28]:
print(EBC_venues.shape)
EBC_venues.head()

(1525, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Albion Gardens,43.741665,-79.584543,Shoppers Drug Mart,43.741685,-79.584487,Pharmacy
1,Albion Gardens,43.741665,-79.584543,Popeyes Louisiana Kitchen,43.741202,-79.584545,Fried Chicken Joint
2,Albion Gardens,43.741665,-79.584543,The Beer Store,43.741694,-79.584373,Beer Store
3,Albion Gardens,43.741665,-79.584543,Subway,43.742645,-79.589643,Sandwich Place
4,Albion Gardens,43.741665,-79.584543,Sheriff's No Frills,43.741696,-79.584379,Grocery Store


Let's check how many venues were returned for each neighborhood


In [29]:
EBC_venues.groupby('Neighborhood').count().min()

Neighborhood Latitude     5
Neighborhood Longitude    5
Venue                     5
Venue Latitude            5
Venue Longitude           5
Venue Category            5
dtype: int64

#### Let's find out how many unique categories can be curated from all the returned venues


In [30]:
print('There are {} uniques categories.'.format(len(EBC_venues['Venue Category'].unique())))

There are 201 uniques categories.


<a id='item3'></a>


## Analyze Each Neighborhood


In [31]:
# one hot encoding
EBC_onehot = pd.get_dummies(EBC_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
EBC_onehot['Neighborhood'] = EBC_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [EBC_onehot.columns[-1]] + list(EBC_onehot.columns[:-1])
EBC_onehot = EBC_onehot[fixed_columns]

EBC_onehot.head()

,Yoga Studio,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Optical Shop,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool Hall,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Albion Gardens,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Albion Gardens,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Albion Gardens,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.


In [32]:
EBC_onehot.shape

(1525, 201)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [33]:
EBC_grouped = EBC_onehot.groupby('Neighborhood').mean().reset_index()
EBC_grouped

,Neighborhood,Yoga Studio,American Restaurant,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Office,Optical Shop,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool Hall,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Track,Trail,Train Station,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store
0,Albion Gardens,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.066667,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.066667,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.066667,0.00,0.00,0.00,0.000000,0

#### Let's confirm the new size


In [34]:
EBC_grouped.shape

(45, 201)

In [35]:
num_top_venues = 5

for hood in EBC_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = EBC_grouped[EBC_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albion Gardens----
            venue  freq
0     Pizza Place  0.20
1   Grocery Store  0.13
2  Hardware Store  0.07
3    Liquor Store  0.07
4            Park  0.07


----Alderwood----
               venue  freq
0     Discount Store  0.12
1        Pizza Place  0.12
2  Convenience Store  0.12
3           Pharmacy  0.06
4      Grocery Store  0.06


----Bloordale Gardens----
                  venue  freq
0           Coffee Shop  0.12
1  Fast Food Restaurant  0.08
2        Sandwich Place  0.08
3              Pharmacy  0.04
4           Supermarket  0.04


----Cloverdale----
                  venue  freq
0  Fast Food Restaurant  0.10
1        Sandwich Place  0.08
2           Coffee Shop  0.05
3           Supermarket  0.05
4            Restaurant  0.03


----Eringate----
                         venue  freq
0  Eastern European Restaurant  0.17
1            Convenience Store  0.17
2                         Park  0.17
3           Chinese Restaurant  0.17
4                  Pizza Place  0.17



In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = EBC_grouped['Neighborhood']

for ind in np.arange(EBC_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(EBC_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Albion Gardens,Pizza Place,Grocery Store,Park,Caribbean Restaurant,Sushi Restaurant
1,Alderwood,Discount Store,Convenience Store,Pizza Place,Donut Shop,Park
2,Bloordale Gardens,Coffee Shop,Sandwich Place,Fast Food Restaurant,Donut Shop,Deli / Bodega
3,Cloverdale,Fast Food Restaurant,Sandwich Place,Supermarket,Coffee Shop,Department Store
4,Eringate,Coffee Shop,Park,Convenience Store,Pizza Place,Eastern European Restaurant


## Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 4 clusters.


In [38]:
# set number of clusters
kclusters = 4

EBC_grouped_clustering = EBC_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(EBC_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 0, 0, 1, 2, 0, 1, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.


In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

EBC_merged = EBC

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
EBC_merged = EBC_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

EBC_merged.head() # check the last columns!

,Borough,Neighbourhood,LAT,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Etobicoke,Albion Gardens,43.741665,-79.584543,3,Pizza Place,Grocery Store,Park,Caribbean Restaurant,Sushi Restaurant
1,Etobicoke,Alderwood,43.601717,-79.545232,1,Discount Store,Convenience Store,Pizza Place,Donut Shop,Park
3,Etobicoke,Bloordale Gardens,43.635317,-79.563674,0,Coffee Shop,Sandwich Place,Fast Food Restaurant,Donut Shop,Deli / Bodega
4,Etobicoke,Cloverdale,43.633637,-79.549745,0,Fast Food Restaurant,Sandwich Place,Supermarket,Coffee Shop,Department Store
5,Etobicoke,Eringate,43.662273,-79.576516,1,Coffee Shop,Park,Convenience Store,Pizza Place,Eastern European Restaurant


Finally, let's visualize the resulting clusters


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(EBC_merged['LAT'], EBC_merged['Long'], EBC_merged['Neighbourhood'], EBC_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [41]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 0, EBC_merged.columns[[1] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Bloordale Gardens,Coffee Shop,Sandwich Place,Fast Food Restaurant,Donut Shop,Deli / Bodega
4,Cloverdale,Fast Food Restaurant,Sandwich Place,Supermarket,Coffee Shop,Department Store
8,Humber Valley Village,Pharmacy,Grocery Store,Shopping Mall,Park,Bank
10,Islington,Coffee Shop,Pub,Italian Restaurant,Sandwich Place,Fast Food Restaurant
11,Islington Avenue,Restaurant,Brewery,Coffee Shop,Sushi Restaurant,Sporting Goods Shop
14,Kingsview Village,Coffee Shop,Intersection,Supermarket,Breakfast Spot,Sporting Goods Shop
15,Kingsway Park South East,Coffee Shop,Pub,Sushi Restaurant,Pizza Place,Breakfast Spot
16,Kingsway Park South West,Coffee Shop,Pub,Sushi Restaurant,Pizza Place,Breakfast Spot
17,Long Branch,Bar,Hobby Shop,Gas Station,Sandwich Place,Burger Joint
19,Martin Grove,Coffee Shop,Café,Gastropub,Hotel,Restaurant


#### Cluster 2


In [42]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 1, EBC_merged.columns[[1] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Alderwood,Discount Store,Convenience Store,Pizza Place,Donut Shop,Park
5,Eringate,Coffee Shop,Park,Convenience Store,Pizza Place,Eastern European Restaurant
9,Humbergate,Park,Italian Restaurant,Harbor / Marina,Pharmacy,Bank
13,King's Mill Park,Coffee Shop,Pizza Place,Café,Park,Italian Restaurant
20,Martin Grove Gardens,Pharmacy,Pizza Place,Sandwich Place,Paintball Field,Bus Line
28,Old Burnhamthorpe,Gas Station,Pharmacy,Liquor Store,Coffee Shop,Shopping Plaza
29,Old Mill North,Park,Coffee Shop,Pizza Place,Pub,River
30,Old Mill South,Park,Coffee Shop,Pizza Place,Pub,River
32,Richview Gardens,Beer Store,Shopping Mall,Sandwich Place,Pizza Place,Pharmacy
35,Silverstone,Shopping Mall,Fast Food Restaurant,Park,Indian Restaurant,Coffee Shop


#### Cluster 3


In [43]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 2, EBC_merged.columns[[1] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Humber Bay,Park,Shopping Mall,River,Harbor / Marina,Women's Store


In [44]:
EBC_merged.loc[EBC_merged['Cluster Labels'] == 3, EBC_merged.columns[[1] + list(range(5, EBC_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Albion Gardens,Pizza Place,Grocery Store,Park,Caribbean Restaurant,Sushi Restaurant
12,Jamestown,Pizza Place,Grocery Store,Park,Sushi Restaurant,Fast Food Restaurant
18,Markland Wood,Pizza Place,Baseball Field,Park,Gas Station,Discount Store
